In [ ]:
import os
import sys
import time
import serial
import math
import numpy as np

# Adding the src folder in the current directory as it contains the script
# with the Thymio class
sys.path.insert(0, os.path.join(os.getcwd(), 'utils'))
sys.path.insert(0, os.path.join(os.getcwd(), 'ekf'))

#print(os.getcwd())
from Thymio import Thymio
from motion import Robot
from motion import RepeatedTimer

#/dev/cu.usbmodem141101	/dev/cu.usbmodem141401
th = Thymio.serial(port="/dev/cu.usbmodem141401", refreshing_rate=0.1) #/dev/ttyACM0 for linux
my_th = Robot(th)

time.sleep(3) # To make sure the Thymio has had time to connect

variables = th.variable_description()

print(variables[0])


In [ ]:

from ekf import *
from IPython.display import clear_output



# State Vector [x y yaw v]'

xEst = np.zeros((5, 1))
xTrue = np.zeros((5, 1))
PEst = np.eye(5)

xDR = np.zeros((5, 1))  # Dead reckoning

# history
hxEst = xEst
hxTrue = xTrue
hxDR = xTrue
hz = np.zeros((3, 1))

def repeated_function():
    #global curr_speed, left, right
    global xEst, xTrue, PEst, xDR, hxEst, hxTrue, hxDR, hz

    curr_speed = my_th.get_speed()
    left, right = curr_speed[0], curr_speed[1]
    print(curr_speed)
    u = calc_input(left, right)

    xTrue, z, xDR, ud = observation(xTrue, xDR, u)

    xEst, PEst = ekf_estimation(xEst, PEst, z, ud)

    my_th.set_position([xEst[0][0], xEst[1][0], np.rad2deg(xEst[2][0])])
    print('My position:', my_th.get_position())

    # store data history
    hxEst = np.hstack((hxEst, xEst))
    hxDR = np.hstack((hxDR, xDR))
    hxTrue = np.hstack((hxTrue, xTrue))
    hz = np.hstack((hz, z))

    plt.cla()
    # for stopping simulation with the esc key.
    plt.gcf().canvas.mpl_connect('key_release_event',
            lambda event: [exit(0) if event.key == 'escape' else None])
    plt.plot(hz[0, :], hz[1, :], ".g")
    plt.plot(hxTrue[0, :].flatten(),
                hxTrue[1, :].flatten(), "-b")
    #plt.plot(hxDR[0, :].flatten(),
    #         hxDR[1, :].flatten(), "-k")
    plt.plot(hxEst[0, :].flatten(),
                hxEst[1, :].flatten(), "-r")
    #plot_covariance_ellipse(xEst, PEst)
    plt.axis('equal')
    plt.grid(True)
    #plt.xlim(0, 0.2)
    #plt.ylim(-0.05,0.05)

    plt.xlabel('x')
    plt.ylabel('y')
    #plt.pause(0.001)

rt_speed = RepeatedTimer(0.1, repeated_function)

my_th.set_speed(100)
my_th.set_position([0,0,0])
my_th.move_to_target([0,50])
my_th.move_to_target([50,50])
my_th.move_to_target([50,100])
my_th.move_to_target([70,100])
my_th.move_to_target([80,140])

print('My position:', my_th.get_position())

# my_th.go_straight(200)
# my_th.turn(45)
# my_th.go_straight(100)
# my_th.turn(90)
# my_th.go_straight(100)


rt_speed.stop()
my_th.stop()


In [ ]:


def repeated_function():
    global curr_speed, left, right
    curr_speed = my_th.get_speed() * 0.31573
    left, right = curr_speed[0], curr_speed[1]
    print(left/95 - right/95)

rt_speed = RepeatedTimer(1, repeated_function)

my_th.set_speed(100)
my_th.go_straight(200)

my_th.turn(90)
my_th.go_straight(200)

time = 0.0

# State Vector [x y yaw v]'
xEst = np.zeros((4, 1))
xTrue = np.zeros((4, 1))
PEst = np.eye(4)

xDR = np.zeros((4, 1))  # Dead reckoning

# history
hxEst = xEst
hxTrue = xTrue
hxDR = xTrue
hz = np.zeros((2, 1))

while SIM_TIME >= time:
    time += DT
    u = calc_input(left, right)
    print(u)
    print(time)
    xTrue, z, xDR, ud = observation(xTrue, xDR, u)

    xEst, PEst = ekf_estimation(xEst, PEst, z, ud)

    # store data history
    hxEst = np.hstack((hxEst, xEst))
    hxDR = np.hstack((hxDR, xDR))
    hxTrue = np.hstack((hxTrue, xTrue))
    hz = np.hstack((hz, z))

    if show_animation:
        #clear_output(wait=True)
        plt.cla()
        # for stopping simulation with the esc key.
        plt.gcf().canvas.mpl_connect('key_release_event',
                lambda event: [exit(0) if event.key == 'escape' else None])
        plt.plot(hz[0, :], hz[1, :], ".g")
        plt.plot(hxTrue[0, :].flatten(),
                    hxTrue[1, :].flatten(), "-b")
        #plt.plot(hxDR[0, :].flatten(),
        #         hxDR[1, :].flatten(), "-k")
        plt.plot(hxEst[0, :].flatten(),
                    hxEst[1, :].flatten(), "-r")
        #plot_covariance_ellipse(xEst, PEst)
        plt.axis('equal')
        plt.grid(True)
        #plt.xlim(0, 0.2)
        #plt.ylim(-0.05,0.05)

        plt.xlabel('x')
        plt.ylabel('y')
        #plt.pause(0.001)



#my_th.move_to_target([40,40])
#my_th.move_to_target([100,220])


rt_speed.stop()
my_th.stop()


In [ ]:
rt_speed.stop()
my_th.stop()

In [ ]:
my_th.rt_speed.start()
test = my_th.speed_array[-1]
#print(test)
my_th.set_speed(100)
#test = my_th.get_speed()
#print(test)
#print(len(test))
#my_th.forward(300)
#my_th.turn(360)
#my_th.go_straight(500)
my_th.turn(360)
#my_th.turn(-180)


In [ ]:
my_th.rt_speed.stop()
my_th.stop()

In [ ]:
print(test)

In [ ]:
my_th.rt_speed.start()

my_th.move_to_target([100,0])
my_th.move_to_target([100,50])
my_th.set_position([0,0,0])

my_th.rt_speed.stop()
my_th.stop()

In [ ]:
from ekf import *
%matplotlib inline
from IPython.display import clear_output

def main():
    #print(__file__ + " start!!")
    print("start!")

    time = 0.0

    # State Vector [x y yaw v]'
    xEst = np.zeros((4, 1))
    xTrue = np.zeros((4, 1))
    PEst = np.eye(4)

    xDR = np.zeros((4, 1))  # Dead reckoning

    # history
    hxEst = xEst
    hxTrue = xTrue
    hxDR = xTrue
    hz = np.zeros((2, 1))

    while SIM_TIME >= time:
        time += DT
        u = calc_input()

        xTrue, z, xDR, ud = observation(xTrue, xDR, u)

        xEst, PEst = ekf_estimation(xEst, PEst, z, ud)

        # store data history
        hxEst = np.hstack((hxEst, xEst))
        hxDR = np.hstack((hxDR, xDR))
        hxTrue = np.hstack((hxTrue, xTrue))
        hz = np.hstack((hz, z))

        if show_animation:
            clear_output(wait=True)
            plt.cla()
            # for stopping simulation with the esc key.
            plt.gcf().canvas.mpl_connect('key_release_event',
                    lambda event: [exit(0) if event.key == 'escape' else None])
            plt.plot(hz[0, :], hz[1, :], ".g")
            plt.plot(hxTrue[0, :].flatten(),
                     hxTrue[1, :].flatten(), "-b")
            #plt.plot(hxDR[0, :].flatten(),
            #         hxDR[1, :].flatten(), "-k")
            plt.plot(hxEst[0, :].flatten(),
                     hxEst[1, :].flatten(), "-r")
            #plot_covariance_ellipse(xEst, PEst)
            #plt.axis("equal")
            plt.grid(True)
            plt.xlim(0, 0.2)
            plt.ylim(-0.05,0.05)

            plt.xlabel('x')
            plt.ylabel('y')
            plt.pause(0.001)

if __name__ == '__main__':
    main()

In [ ]:

my_th.go_straight(500)

"""
my_th.forward(100)
my_th.turn(180)
my_th.turn(360)
my_th.turn(-720)
my_th.move_to_target([0,1])
my_th.move_to_target([12,33])

full_path = np.array([[0,1,1,2,3,4,5,5,5,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,7,7,7,7,7,8,9,10,11,12,13,14,15,16,17,18,19,20],
 [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,38,39,39,39,40]])

full_path = full_path.T

for i in range(len(full_path)):
    my_th.move_to_target([full_path[i,0],full_path[i,1]])
    pos_th = my_th.get_position()
    print("Thymio position: {}".format(pos_th))"""
conversion = 0.31573
speed = 31.573 #[mm/s]

print(my_th.get_speed())
my_th.stop()


In [ ]:
my_th.stop()